This notebook evaluates Saliency Maps with UCF-50 frame using the tf-keras-vis implementation. 

This notebook is based on the example presented in tf-keras-vis (Kubota, Y. (2021). tf-keras-vis (Version 0.8.1) [Computer software]. https://keisen.github.io/tf-keras-vis-docs/)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
import os
import cv2

Load frames and masks

In [ ]:
!cp -R /content/gdrive/MyDrive/masked_images_backup/masks /
!cp -R /content/gdrive/MyDrive/video_frames/ /

In [ ]:
!pip install tf-keras-vis

     |████████████████████████████████| 53 kB 1.5 MB/s 


In [ ]:
import tf_keras_vis

In [ ]:
model = keras.models.load_model('/content/gdrive/MyDrive/resnet50_v2_transferLearned_model')

In [ ]:
from keras.preprocessing.image import load_img
from keras.applications.resnet_v2 import preprocess_input

In [ ]:
from PIL import Image

In [ ]:
classes_list = ['PizzaTossing', 'PlayingGuitar', 'PushUps', 'SalsaSpin']

In [ ]:
def prepare_data(class_name, dir_name, check_preds=False):

  images =[]
  file_names =[]
  masks = []

  masks_root = '/masks'
  images_root ='/video_frames'
  
  
  for mask_name in os.listdir(os.path.join(masks_root, class_name, dir_name, 'masks')):
    image_name = mask_name.replace('png','jpg')
    image_path = os.path.join(images_root,class_name,dir_name,image_name)
    mask_path = os.path.join(masks_root, class_name, dir_name, 'masks', mask_name)

    im = load_img(image_path,target_size=(100,100))
    im = np.array(im)
    x = np.asarray([im])
    x = preprocess_input(x)
    pred = model.predict(x)

    if not os.path.exists('/saliency_maps'):
      os.mkdir('/saliency_maps')
    if not os.path.exists(os.path.join('/saliency_maps',class_name)):
      os.mkdir(os.path.join('/saliency_maps',class_name))
    if not os.path.exists(os.path.join('/saliency_maps', class_name, dir_name)):
      os.mkdir(os.path.join('/saliency_maps', class_name, dir_name))
    

    if classes_list[np.argmax(pred[0])] == class_name or check_preds:
      file_names.append(image_path)



      msk = np.array(Image.open(mask_path).resize(im.shape[1::-1], Image.BILINEAR))
      msk = msk/255

      images.append(im)
      masks.append(msk)
    

  images = np.asarray(images)

  # Preparing input data
  X = preprocess_input(images)
  

  return images, masks, file_names, X

In [ ]:
from tf_keras_vis.utils.model_modifiers import ReplaceToLinear
from tf_keras_vis.utils.scores import CategoricalScore
from keras import backend as K
from tf_keras_vis.saliency import Saliency
import time

In [ ]:
def create_saliency_maps(class_name, dir_name, check_preds=False):
  images, masks, file_names, X = prepare_data(class_name, dir_name, check_preds=check_preds)
  replace2linear = ReplaceToLinear()
  # 0 = PizzaTossing, 1 = PlayingGuiar, 2 = PushUps, 3 = SalsaSpin.
  for index, name in enumerate(classes_list):
    if name==class_name:
      score = CategoricalScore([index])
  start = time.time()
  saliency = Saliency(model,
                    model_modifier=replace2linear,
                    clone=True)
  
  # Generate saliency map
  saliency_map = saliency(score, X)
  duration = time.time()-start

  for index, map in enumerate(saliency_map):
    path = file_names[index].replace('video_frames','saliency_maps')

    # Render
    f, ax = plt.subplots(nrows=1, ncols=1, figsize=(4, 4))
    ax.set_title(class_name, fontsize=16)
    ax.imshow(images[index])
    ax.imshow(map, cmap='jet', alpha=0.5)
    ax.axis('off')
    plt.tight_layout()
    plt.savefig(path)
    plt.close()
    #plt.show()

  return saliency_map, images, masks, file_names, duration


In [ ]:
def evaluate_maps(saliency_maps, masks, duration):
  duration = duration/ len(saliency_maps)
  duration_list = []
  duration_list.extend([duration]*len(saliency_maps))

  map_weight_list = []
  mask_weight_list = []
  for index, map in enumerate(saliency_maps):
    map_weight = np.sum(map)
    dst = map * masks[index]
    mask_weight = np.sum(dst)

    map_weight_list.append(map_weight)
    mask_weight_list.append(mask_weight)

  return map_weight_list, mask_weight_list, duration_list



In [ ]:
pizza = [('PizzaTossing','v_PizzaTossing_g02_c02')]
salsa = [('SalsaSpin','v_SalsaSpin_g04_c02'),('SalsaSpin','v_SalsaSpin_g11_c02'),('SalsaSpin','v_SalsaSpin_g18_c04')]
guitar = [('PlayingGuitar', 'v_PlayingGuitar_g06_c05'), ('PlayingGuitar', 'v_PlayingGuitar_g11_c02'),('PlayingGuitar','v_PlayingGuitar_g22_c03')]
pushups = [('PushUps', 'v_PushUps_g11_c03'), ('PushUps','v_PushUps_g14_c04'),('PushUps','v_PushUps_g18_c02')]

#all_classes = [pizza, salsa, guitar, pushups]

In [ ]:
map_weights = []
mask_weights = []
durations = []
all_classes = [pizza, salsa]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_saliency_maps(name, dir)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
if not os.path.exists('/content/gdrive/MyDrive/ResNet50'):
  os.mkdir('/content/gdrive/MyDrive/ResNet50')
!cp -R /saliency_maps /content/gdrive/MyDrive/ResNet50/

map_weights = []
mask_weights = []
durations = []
all_classes = [pizza]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_saliency_maps(name, dir)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')

map_weights = []
mask_weights = []
durations = []
all_classes = [salsa]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_saliency_maps(name, dir)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')


map_weights = []
mask_weights = []
durations = []
all_classes = [pushups]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_saliency_maps(name, dir, check_preds = True)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
if not os.path.exists('/content/gdrive/MyDrive/ResNet50/PushUps'):
  os.mkdir('/content/gdrive/MyDrive/ResNet50/PushUps')
!cp -R /saliency_maps /content/gdrive/MyDrive/ResNet50/PushUps/

map_weights = []
mask_weights = []
durations = []
all_classes = [guitar]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_saliency_maps(name, dir, check_preds=True)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
if not os.path.exists('/content/gdrive/MyDrive/ResNet50/PlayingGuitar'):
  os.mkdir('/content/gdrive/MyDrive/ResNet50/PlayingGuitar')
!cp -R /saliency_maps /content/gdrive/MyDrive/ResNet50/PlayingGuitar/


map_weights = []
mask_weights = []
durations = []
all_classes = [pizza]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_saliency_maps(name, dir, check_preds=True)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
if not os.path.exists('/content/gdrive/MyDrive/ResNet50/PizzaTossing'):
  os.mkdir('/content/gdrive/MyDrive/ResNet50/PizzaTossing')
!cp -R /saliency_maps /content/gdrive/MyDrive/ResNet50/PizzaTossing/


map_weights = []
mask_weights = []
durations = []
all_classes = [salsa]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_saliency_maps(name, dir, check_preds=True)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')
if not os.path.exists('/content/gdrive/MyDrive/ResNet50/SalsaSpin'):
  os.mkdir('/content/gdrive/MyDrive/ResNet50/SalsaSpin')
!cp -R /saliency_maps /content/gdrive/MyDrive/ResNet50/SalsaSpin/


map_weights = []
mask_weights = []
durations = []
all_classes = [pizza, salsa, guitar, pushups]
for c in all_classes:

  for name, dir in c:
    #print('Evaluating ', name, dir)
    heatmap, _, mask, _ , duration = create_saliency_maps(name, dir, check_preds=True)
    map_w, mask_w, dur = evaluate_maps(heatmap, mask, duration)
    map_weights.extend(map_w)
    mask_weights.extend(mask_w)
    durations.extend(dur)
    #print('Done\n')

print('Average mask weight: ',sum(mask_weights)/len(mask_weights))
print('Standard deviation: ', np.std(mask_weights))
print('Average total weight: ', sum(map_weights)/len(map_weights))
print('Standard deviation: ', np.std(map_weights))
print('Average computing time: ', sum(durations)/len(durations))
print('Standard deviation: ', np.std(durations))

print('Mask weight percentage: ',((sum(mask_weights)/len(mask_weights))/(sum(map_weights)/len(mask_weights)))*100,'%\n')

Average mask weight:  125.71549475097443
Standard deviation:  45.06113345702835
Average total weight:  946.7120882670084
Standard deviation:  209.55234
Average computing time:  0.3043489423063063
Standard deviation:  0.1433954042113415
Mask weight percentage:  13.279168641556199 %

Average mask weight:  109.06198011676877
Standard deviation:  27.261311027101762
Average total weight:  972.587232236509
Standard deviation:  218.85225
Average computing time:  0.22458517109906262
Standard deviation:  1.1102230246251565e-16
Mask weight percentage:  11.213593650204079 %

Average mask weight:  175.67603865359172
Standard deviation:  50.640015411913964
Average total weight:  869.086656358507
Standard deviation:  154.67888
Average computing time:  0.5006386306550766
Standard deviation:  0.1122254540729494
Mask weight percentage:  20.213869050720255 %

Average mask weight:  309.315324578014
Standard deviation:  103.01324322889488
Average total weight:  1004.7999014262281
Standard deviation:  209.